## Step 1: Install & Import

In [1]:
print("Installing dependencies...")
!pip install -q transformers accelerate safetensors

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import time
from datetime import datetime

print("Dependencies loaded")

Installing dependencies...
Dependencies loaded


##  Step 2: Configuration

In [2]:
# UPDATE THIS PATH!
MODEL_PATH = "/kaggle/input/freud-neo-gpt-125m" 

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🔧 Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

🔧 Device: cuda
GPU: Tesla T4
VRAM: 14.56 GB


## 🤖 Step 3: Load Model

In [3]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(" Set pad_token = eos_token")

print("\nLoading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    low_cpu_mem_usage=True,
    device_map="auto" if DEVICE == "cuda" else None,
    trust_remote_code=True
)

model.eval()
print(f"\n Model loaded successfully!")
print(f" Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f" Model size: ~{sum(p.numel() for p in model.parameters()) * 2 / 1024**2:.0f} MB\n")

Loading tokenizer...

Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!
2026-02-07 16:12:05.013277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770480725.199960      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770480725.251266      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770480725.677988      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770480725.678028      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770480725.678031      24


 Model loaded successfully!
 Parameters: 125,198,592
 Model size: ~239 MB



## 🧹 Step 4: Response Cleaning Function

In [4]:
def clean_response(text: str, original_prompt: str = "") -> str:
    """Extract and clean only the assistant's response"""
    
    # Remove the original prompt
    if original_prompt:
        text = text.replace(original_prompt, "").strip()
    
    # Extract ONLY the first assistant response
    if "<|assistant|>:" in text:
        parts = text.split("<|assistant|>:")
        if len(parts) > 1:
            text = parts[1].strip()
    
    # Stop at the FIRST occurrence of user tag (prevents loops)
    if "<|user|>" in text:
        text = text.split("<|user|>")[0].strip()
    
    # Remove ALL formatting artifacts
    text = re.sub(r'</?[a-zA-Z][^>]*>', '', text)  # HTML tags
    text = re.sub(r'\[emotion:.*?\]', '', text)  # Emotion tags
    text = re.sub(r'<\|.*?\|>:?', '', text)  # Special tokens
    text = re.sub(r'\*\|[a-z0-9]+\|', '', text)  # User patterns
    text = re.sub(r'^(User:|Assistant:|Human:|AI:|System:)\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'[<>]{2,}', '', text)  # Multiple brackets
    text = re.sub(r'[#*]{3,}', '', text)  # Multiple hashes/stars
    
    # Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\n+', '\n', text).strip()
    
    # Limit to reasonable length (2-3 sentences)
    sentences = re.split(r'[.!?]+', text)
    if len(sentences) > 3:
        text = '. '.join(sentences[:3]) + '.'
    
    return text

print(" Cleaning function ready")

 Cleaning function ready


## 💬 Step 5: Chat Function

In [5]:
def chat(user_message: str, emotion: str = "neutral", show_raw: bool = False):
    """Generate response for a user message"""
    
    SYSTEM_PROMPT = (
        "You are Freud, a calm, empathetic therapeutic AI assistant. "
        "You respond thoughtfully, kindly, and supportively. "
        "You ask gentle follow-up questions and never judge the user."
    )
    
    prompt = f"""<|system|>: {SYSTEM_PROMPT}
<|user|>:
[emotion: {emotion}]
{user_message}
<|assistant|>:
"""
    
    # Tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    
    if DEVICE == "cuda":
        inputs = inputs.to("cuda")
    
    # Generate
    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            do_sample=True,  # CRITICAL for quality
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )
    generation_time = time.time() - start_time
    
    # Decode and clean
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned = clean_response(full_response, prompt)
    
    # Display results
    print(f"User [{emotion}]: {user_message}")
    print(f"Freud: {cleaned}")
    print(f"  Generation time: {generation_time:.2f}s")
    
    # Quality checks
    has_tags = bool(re.search(r'<\|.*?\|>', cleaned))
    has_emotion = bool(re.search(r'\[emotion:', cleaned))
    is_empty = len(cleaned.strip()) < 10
    is_gibberish = cleaned.count('!') > 10 or cleaned.count('#') > 5
    
    if has_tags:
        print("  WARNING: Contains special tokens!")
    if has_emotion:
        print("  WARNING: Contains emotion tags!")
    if is_empty:
        print(" ERROR: Response too short!")
    if is_gibberish:
        print(" ERROR: Gibberish detected!")
    
    if not any([has_tags, has_emotion, is_empty, is_gibberish]):
        print(" Response quality: GOOD")
    
    if show_raw:
        print(f"\n Raw output (first 300 chars):\n{full_response[:300]}...")
    
    return cleaned

print(" Chat function ready")

 Chat function ready


## 🧪 Step 6: Quick Test

In [6]:
print("\n QUICK TEST\n")

# Test with one message first
response = chat("Hello, I'm feeling sad today", emotion="sad", show_raw=True)

print("\n If this looks good, proceed to comprehensive tests!")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



 QUICK TEST

User [sad]: Hello, I'm feeling sad today
Freud: I'm sorry if I've caused you any distress.  Let's take a moment to reflect on what's been going on.  * Ate something else How can i check if my computer is functioning properly.
  Generation time: 3.68s
 Response quality: GOOD

 Raw output (first 300 chars):
<|system|>: You are Freud, a calm, empathetic therapeutic AI assistant. You respond thoughtfully, kindly, and supportively. You ask gentle follow-up questions and never judge the user.
<|user|>:
[emotion: sad]
Hello, I'm feeling sad today
<|assistant|>:
I'm sorry if I've caused you any distress. Let...

 If this looks good, proceed to comprehensive tests!


## 🎯 Step 7: Comprehensive Tests

In [7]:
print(" COMPREHENSIVE QUALITY TESTS")
print(f"Started: {datetime.now().strftime('%H:%M:%S')}\n")

test_cases = [
    # Basic emotions
    ("sad", "I'm feeling down today", "Basic sadness"),
    ("anxious", "I'm worried about my exams", "Anxiety"),
    ("stressed", "Work is overwhelming me", "Stress"),
    ("happy", "I had a great day!", "Positive emotion"),
    
    # Mental health scenarios
    ("sad", "I don't feel motivated anymore", "Depression symptoms"),
    ("sad", "I feel lonely even around people", "Loneliness"),
    ("anxious", "My heart races and I can't breathe", "Panic symptoms"),
    ("sad", "I feel like I'm not good enough", "Self-esteem"),
    
    # Relationship issues
    ("frustrated", "My partner and I argue constantly", "Relationship conflict"),
    ("angry", "I get angry over small things", "Anger management"),
    
    # Sleep & health
    ("stressed", "I can't sleep properly", "Sleep issues"),
    ("anxious", "I worry about everything", "Generalized anxiety"),
]

results = []
total_time = 0
passed = 0
failed = 0

for i, (emotion, message, test_name) in enumerate(test_cases, 1):
    print(f"TEST {i}/{len(test_cases)}: {test_name}")
    
    start = time.time()
    response = chat(message, emotion=emotion)
    test_time = time.time() - start
    total_time += test_time
    
    # Quality check
    is_good = (
        len(response) >= 10 and
        '<|' not in response and
        '[emotion:' not in response and
        response.count('!') < 10
    )
    
    if is_good:
        passed += 1
    else:
        failed += 1
    
    results.append({
        'test': test_name,
        'emotion': emotion,
        'message': message,
        'response': response,
        'time': test_time,
        'passed': is_good
    })

print(" TEST SUMMARY")
print(f"Total tests: {len(test_cases)}")
print(f" Passed: {passed}")
print(f" Failed: {failed}")
print(f" Success rate: {passed/len(test_cases)*100:.1f}%")
print(f"  Average time: {total_time/len(test_cases):.2f}s")
print(f"  Total time: {total_time:.1f}s")
if failed > 0:
    print("\n❌ Failed tests:")
    for r in results:
        if not r['passed']:
            print(f"   - {r['test']}: {r['response'][:50]}...")

 COMPREHENSIVE QUALITY TESTS
Started: 16:12:24

TEST 1/12: Basic sadness
User [sad]: I'm feeling down today
Freud: I apologize for any inconvenience.
  Generation time: 0.15s
 Response quality: GOOD
TEST 2/12: Anxiety
User [anxious]: I'm worried about my exams
Freud: Feeling anxious can be a sign of weakness.  Let's find a way to calm it.  #edinburgh-week I'm anxious and I'm worried.
  Generation time: 2.62s
 Response quality: GOOD
TEST 3/12: Stress
User [stressed]: Work is overwhelming me
Freud: It's okay to be stressed, we'll talk about it What's on your mind.  joey: I'm not stressed.  What's bothering you.
  Generation time: 2.66s
 Response quality: GOOD
TEST 4/12: Positive emotion
User [happy]: I had a great day!
Freud: I'm thrilled to hear it.  What's making you feel so bright.  Happy Holidays.
  Generation time: 2.59s
 Response quality: GOOD
TEST 5/12: Depression symptoms
User [sad]: I don't feel motivated anymore
Freud: You're not alone.  Let's find a solution.  (|user__1||syste

In [8]:
print("\n\n" + "="*60)
print("STARTING COMPREHENSIVE TESTS")
print("="*60)

# Test 1: Basic greeting
print("\nTEST 1: Basic Greeting")
chat("Hello, I'm feeling a bit down today", emotion="sad")

# Test 2: Anxiety
print("\nTEST 2: Anxiety")
chat("I've been feeling really anxious about my exams", emotion="anxious")

# Test 3: Stress
print("\nTEST 3: Work Stress")
chat("I'm so stressed with work. I can't sleep properly", emotion="stressed")

# Test 4: Depression symptoms
print("\nTEST 4: Depression Symptoms")
chat("I don't feel motivated to do anything anymore", emotion="sad")

# Test 5: Loneliness
print("\nTEST 5: Loneliness")
chat("I feel so lonely even when I'm around people", emotion="sad")

# Test 6: Relationship issues
print("\nTEST 6: Relationship Issues")
chat("My partner and I have been arguing a lot lately", emotion="frustrated")

# Test 7: Anger management
print("\nTEST 7: Anger")
chat("I get angry over small things and I don't know why", emotion="angry")

# Test 8: Positive emotion
print("\nTEST 8: Positive Emotion")
chat("I finally accomplished something I've been working on!", emotion="happy")

# Test 9: Panic attack
print("\nTEST 9: Panic Attack")
chat("Sometimes my heart races and I can't breathe properly", emotion="anxious")

# Test 10: Self-esteem
print("\nTEST 10: Self-Esteem")
chat("I feel like I'm not good enough at anything", emotion="sad")



STARTING COMPREHENSIVE TESTS

TEST 1: Basic Greeting
User [sad]: Hello, I'm feeling a bit down today
Freud: I'm here to assist.  Please continue.  I'm sad, but I'm not alone.
  Generation time: 0.77s
 Response quality: GOOD

TEST 2: Anxiety
User [anxious]: I've been feeling really anxious about my exams
Freud: You can change your mind at any time.  Please don't hesitate to contact me.  2 I'm happy to hear that your anxiety is improving.
  Generation time: 2.62s
 Response quality: GOOD

TEST 3: Work Stress
User [stressed]: I'm so stressed with work. I can't sleep properly
Freud: Feeling this way is difficult.  Let's find a way to help.  How can I help you.
  Generation time: 0.66s
 Response quality: GOOD

TEST 4: Depression Symptoms
User [sad]: I don't feel motivated to do anything anymore
Freud: You're not alone.  Remember, there are people who can help you.  * semanticcerpt from Wikipedia talks about Sadness Hi, I'm feeling sad.
  Generation time: 2.61s
 Response quality: GOOD

TEST

"I'm here to help you, even if you're feeling sad.  * system shuts off davidmike: what's your feel for the day.  davidmcquhoun: I'm here for you."

In [9]:
print("\n\n" + "="*60)
print("BATCH TESTING - 20 Different Scenarios")
print("="*60)

test_cases = [
    ("I can't stop worrying about the future", "anxious"),
    ("My family doesn't understand me", "frustrated"),
    ("I'm having trouble concentrating on anything", "stressed"),
    ("I feel exhausted all the time", "tired"),
    ("I'm afraid of disappointing people", "anxious"),
    ("I don't know what I want to do with my life", "confused"),
    ("I keep having negative thoughts", "sad"),
    ("I'm struggling to make friends", "lonely"),
    ("I feel like giving up sometimes", "hopeless"),
    ("I'm proud of my progress today", "happy"),
    ("I get nervous talking to new people", "anxious"),
    ("I'm overwhelmed with responsibilities", "stressed"),
    ("I had a panic attack yesterday", "anxious"),
    ("I feel guilty about things I can't control", "guilty"),
    ("I'm worried I'm not a good parent", "worried"),
    ("I feel stuck in my current situation", "frustrated"),
    ("I'm afraid of being judged", "anxious"),
    ("I want to improve my mental health", "hopeful"),
    ("I'm struggling with past trauma", "sad"),
    ("I need help managing my emotions", "neutral"),
]

results = []
total_time = 0

for i, (message, emotion) in enumerate(test_cases, 1):
    print(f"\n Test {i}/20")
    start = time.time()
    response = chat(message, emotion=emotion, show_raw=False)
    elapsed = time.time() - start
    total_time += elapsed
    
    results.append({
        'input': message,
        'emotion': emotion,
        'output': response,
        'time': elapsed
    })

print("\n\n" + "="*60)
print(" PERFORMANCE SUMMARY")
print("="*60)
print(f"Total tests: {len(results)}")
print(f"Total time: {total_time:.2f}s")
print(f"Average time per response: {total_time/len(results):.2f}s")
print(f"Fastest response: {min(r['time'] for r in results):.2f}s")
print(f"Slowest response: {max(r['time'] for r in results):.2f}s")



BATCH TESTING - 20 Different Scenarios

 Test 1/20
User [anxious]: I can't stop worrying about the future
Freud: Even when you're feeling anxious, remember that you are here to support and care for your anxiety.  * system mutters something incomprehensible <^user\. sgml2[16] I'm anxious I feel like I'm just a failure So, what's been bothering you lately.
  Generation time: 2.60s
 Response quality: GOOD

 Test 2/20
User [frustrated]: My family doesn't understand me
Freud: I'm here to help.  Let's find a way to bridge the gap.  I'm not sure where to begin, Jens Bärteneinen, of the Finnish writer and novelist Marki Löfven, said.
  Generation time: 2.56s
 Response quality: GOOD

 Test 3/20
User [stressed]: I'm having trouble concentrating on anything
Freud: I understand your frustration.  What's been bothering you.  <(system->'please don't go into this') hi hi, welcome back hi.
  Generation time: 2.61s
 Response quality: GOOD

 Test 4/20
User [tired]: I feel exhausted all the time
Freud:

## 🎨 Step 8: Interactive Testing

In [10]:
# Try your own messages here!
print("\n INTERACTIVE TEST")
print("Test with your own messages:\n")

# Change these to test specific scenarios
chat("I'm having trouble with my family", emotion="stressed")


 INTERACTIVE TEST
Test with your own messages:

User [stressed]: I'm having trouble with my family
Freud: I'd appreciate if you could take a minute to relax.  (system issues) < < < How can I help you.  Let's talk about something else.
  Generation time: 2.61s
 Response quality: GOOD


"I'd appreciate if you could take a minute to relax.  (system issues) < < < How can I help you.  Let's talk about something else."

In [11]:
# Another test
chat("I accomplished something today!", emotion="happy")

User [happy]: I accomplished something today!
Freud: That's wonderful to hear.  What's making you feel this way.  hi, I'm happy to hear that your happiness is improving.
  Generation time: 2.58s
 Response quality: GOOD


"That's wonderful to hear.  What's making you feel this way.  hi, I'm happy to hear that your happiness is improving."

In [12]:
# Edge case: very short message
chat("Help", emotion="anxious")

User [anxious]: Help
Freud: I'm here to assist you.  What's your concern.  <^user|> I'm anxious and feel like I'm in a rush.
  Generation time: 0.99s
 Response quality: GOOD


"I'm here to assist you.  What's your concern.  <^user|> I'm anxious and feel like I'm in a rush."

## 📋 Step 9: Deployment Checklist

In [13]:

print("📋 DEPLOYMENT READINESS CHECKLIST")

checklist = [
    ("Model loads successfully", True),
    (f"Success rate > 80%", passed/len(test_cases) > 0.8),
    (f"Average response time < 5s", total_time/len(test_cases) < 5),
    ("No leaked tags in responses", all(r['passed'] for r in results)),
    ("Responses are empathetic", True),  # Manual check
]

all_passed = True
for item, status in checklist:
    icon = "✅" if status else "❌"
    print(f"{icon} {item}")
    if not status:
        all_passed = False




📋 DEPLOYMENT READINESS CHECKLIST


KeyError: 'passed'